<a href="https://colab.research.google.com/github/vjmaurina/Projeto_Imersao_Avaliacao/blob/main/Projeto_Alura_Imersao02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instala a biblioteca google.generativeai, que fornece acesso à API de IA generativa do Google.**

In [ ]:
!pip install -U -q google.generativeai

**Instala a biblioteca chromadb, que fornece uma interface para criar e consultar bancos de dados de embeddings.**

In [ ]:
!pip install -q chromadb

**Importa as bibliotecas necessárias para processamento de texto, criação de banco de dados de embeddings e manipulação de dados.**

In [ ]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

Importa as bibliotecas google.generativeai e google.ai.generativelanguage, que fornecem acesso à API de IA generativa do Google.
from google.colab import userdata

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

**Lista os modelos de IA generativa disponíveis que suportam o método de geração embedContent.**

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

**Define três documentos de texto e os armazena em uma lista.**

In [ ]:
DOCUMENT1 = "Operando o sistema de controle climático Seu Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central. Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite selecionar o modo desejado. Os modos disponíveis são: Automático: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Frio: O carro soprará ar frio para dentro do carro. Aquecimento: O carro soprará ar quente para dentro do carro. Descongelamento: O carro soprará ar quente no para-brisa para descongelá-lo."
DOCUMENT2 = "Seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado. Por exemplo, você pode tocar no ícone Navegação para obter instruções para o seu destino ou tocar no ícone Música para tocar suas músicas favoritas."
DOCUMENT3 = "Mudando de marcha Seu Googlecar tem uma transmissão automática. Para mudar de marcha, basta mover a alavanca de câmbio para a posição desejada. Estacionamento: Esta posição é usada quando você está estacionado. As rodas estão travadas e o carro não pode se mover. Ré: Esta posição é usada para dar ré. Neutro: Esta posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está engatado e não se moverá a menos que você pressione o pedal do acelerador. Dirigir: Esta posição é usada para dirigir para frente. Baixo: Esta posição é usada para dirigir na neve ou em outras condições escorregadias."
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

**Define uma classe personalizada GeminiEmbeddingFunction que herda da classe EmbeddingFunction da biblioteca chromadb. Esta classe é usada para gerar embeddings para documentos de texto usando o modelo de incorporação Gemini.**

In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

**Define uma função create_chroma_db que cria um banco de dados de embeddings usando a biblioteca chromadb. A função leva uma lista de documentos de texto e um nome para o banco de dados como parâmetros.**

In [ ]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

**Cria um banco de dados de embeddings chamado "googlecarsdatabase" usando a função create_chroma_db e os documentos de texto definidos anteriormente.**

In [ ]:
db = create_chroma_db(documents, "googlecarsdatabase")

**Exibe as primeiras três linhas do banco de dados de embeddings como um DataFrame do Pandas.**

In [ ]:
pd.DataFrame(db.peek(3))

**Define uma função get_relevant_passage que leva uma consulta e um banco de dados de embeddings como parâmetros. A função retorna a passagem mais relevante do banco de dados para a consulta.**

In [ ]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

**Recupera a passagem mais relevante do banco de dados de embeddings para a consulta "touch screen features" e a exibe como texto formatado.**

In [ ]:
passage = get_relevant_passage("touch screen features", db)
Markdown(passage)

**Define uma função make_prompt que leva uma consulta e uma passagem relevante como parâmetros. A função retorna um prompt de geração de linguagem natural que pode ser usado para gerar uma resposta à consulta usando a passagem relevante como contexto.**

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""Você é um bot útil e informativo que responde perguntas usando texto da passagem de referência incluída abaixo.
  Certifique-se de responder em uma frase completa, sendo abrangente, incluindo todas as informações relevantes.
  No entanto, você está falando com um público não técnico, por isso certifique-se de quebrar conceitos complicados e
  um tom amigável e conversível.
  Se a passagem é irrelevante para a resposta, você pode ignorá-la.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

**Cria um prompt de geração de linguagem natural para a consulta "Como você usa a tela sensível ao toque no carro do Google?" usando a passagem relevante recuperada anteriormente.**

In [ ]:
query = "Como você usa a tela sensível ao toque no carro do Google?"
prompt = make_prompt(query, passage)
Markdown(prompt)

**Gera uma resposta à consulta usando o modelo de geração de linguagem natural Gemini-pro e o prompt criado anteriormente. A resposta é exibida como texto formatado.**

In [ ]:
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)
Markdown(answer.text)